In [27]:
import cv2
import numpy as np
import os
from tqdm import tqdm

# Get frame


In [18]:

# Path to the folder containing videos
video_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/data/DaiTu/x9.mp4"

# Output folder to save frames
output_folder = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_7"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

    
# Check if the path is a file (to skip any subdirectories)
if os.path.isfile(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get the base name of the video file without extension
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Save the frame as an image file
        frame_filename = os.path.join(output_folder, f"{video_name}_frame_{frame_count}.jpg")
        # if frame_count % 300 == 0:
        cv2.imwrite(frame_filename, frame)
        
        frame_count += 1
        break
    
    # Release the video capture object
    cap.release()

print("Frame extraction completed.")

Frame extraction completed.


# Select point

In [49]:
img_path = '/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_1/x3_frame_0.jpg'
folder_save = img_path[:-14]
print("folder_save: ", folder_save)
image = cv2.imread(img_path)
# image, _, _, _, _ = apply_distortion(img = image)
print("image: ", image.shape)
W = image.shape[1]
H = image.shape[0]

image = cv2.resize(image, (W//2, H//2))
name = img_path.split('/')[-1].split('.')[0]
list_point = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at ({x}, {y})")
        cv2.circle(image, (int(x), int(y)), 3, (0, 0, 255), -1)
        cv2.imshow('Image', image)
        list_point.append(str(round(x/image.shape[1], 3)))
        list_point.append(str(round(y/image.shape[0], 3)))

cv2.namedWindow('Image')
cv2.setMouseCallback('Image', mouse_callback)

cv2.imshow('Image', image)
while True:
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        full_roi = str(';'.join(list_point))
        print(full_roi)
        print(name)
        f = open(os.path.join(folder_save, name + '.txt'), 'w')
        f.write(full_roi)
        f.close()
        break

cv2.destroyAllWindows()

folder_save:  /home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_1/
image:  (1080, 1920, 3)
Clicked at (957, 345)
Clicked at (914, 276)
Clicked at (91, 313)
Clicked at (56, 391)
0.997;0.639;0.952;0.511;0.095;0.58;0.058;0.724
x3_frame_0


# Warp image to one image view

In [38]:
def read_points(point_path, width, height):
    # Read the normalized points from the file
    with open(point_path, 'r') as file:
        points_str = file.read().strip().split(';')
        points = [float(p) for p in points_str]

    # Denormalize the points
    denormalized_points = [
        [points[0] * width, points[1] * height],
        [points[2] * width, points[3] * height],
        [points[4] * width, points[5] * height],
        [points[6] * width, points[7] * height]
    ]
    return np.array(denormalized_points, dtype=np.float32)

def calculate_output_size(points):
    height = max(
        np.linalg.norm(points[0] - points[1]),
        np.linalg.norm(points[3] - points[2])
    )
    width = max(
        np.linalg.norm(points[0] - points[3]),
        np.linalg.norm(points[1] - points[2])
    )

    width = int(width / (height / 300))
    height = 300
    return width, height

In [109]:
# Lấy kích thước của ảnh overview 
# Chiều từ trái sang phải

W1  = 2682
W2 = 2727
W3 = 2722
W4 = 2172
W5 = 2952
W6 = 3197
W7 = 3167

W = W1 + W2 + W3 + W4 + W5 + W6 + W7
H = 300
H_padding = 300
print("[Over view] ", W , " X ", H)

[Over view]  19619  X  300


- Stream 1 ( Từ phải sang)

In [111]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_1/x3_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W, out_height + H_padding],  # Bottom right
    [W, 0 + H_padding],           # Top right
    [W1 + W2 + W3 + W4 + W5 + W6, 0 + H_padding],                       # Top left
    [W1 + W2 + W3 + W4 + W5 + W6, out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_1 = cv2.warpPerspective(image, Homo, (int(W), int(H  + 2 * H_padding)))



# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_1)


width:  3167  height:  300
denormalized_points:  [[1914.24  690.12]
 [1827.84  551.88]
 [ 182.4   626.4 ]
 [ 111.36  781.92]]
Homography Matrix:  [[ 2.83290527e+00  1.87265654e+01  1.50189488e+04]
 [ 1.58430428e-01  3.90509266e+00 -1.97785616e+03]
 [-1.37177229e-05  1.05333283e-03  1.00000000e+00]]


True

- Stream 2 ( Từ phải sang)

In [114]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_2/x4_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W - W7, out_height + H_padding],  # Bottom right
    [W - W7, 0 + H_padding],           # Top right
    [W1 + W2 + W3 + W4 + W5, 0 + H_padding],                       # Top left
    [W1 + W2 + W3 + W4 + W5, out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_2 = cv2.warpPerspective(image, Homo, (int(W), int(H + H_padding * 2)))

# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_2)


width:  3197  height:  300
denormalized_points:  [[1918.08  828.36]
 [1841.28  680.4 ]
 [ 213.12  587.52]
 [ 134.4   735.48]]
Homography Matrix:  [[ 1.38750612e+00  1.44056993e+01  1.17849145e+04]
 [-2.23006047e-01  3.64261496e+00 -1.62760465e+03]
 [-1.06293414e-04  9.74568151e-04  1.00000000e+00]]


True

- Stream 3 ( Từ phải sang)

In [113]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_3/x5_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W - W7 - W6, out_height + H_padding],  # Bottom right
    [W - W7 - W6, 0 + H_padding],           # Top right
    [W1 + W2 + W3 + W4, 0 + H_padding],                       # Top left
    [W1 + W2 + W3 + W4, out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_3 = cv2.warpPerspective(image, Homo, (int(W), int(H + H_padding*2)))

# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_3)


width:  2952  height:  300
denormalized_points:  [[1912.32  762.48]
 [1825.92  600.48]
 [ 178.56  516.24]
 [ 107.52  671.76]]
Homography Matrix:  [[ 1.95692773e+00  1.00401860e+01  9.26992674e+03]
 [-1.67406571e-01  3.22761387e+00 -1.20531596e+03]
 [-5.20402136e-05  8.63958416e-04  1.00000000e+00]]


True

- Stream 4 ( Từ phải sang)

In [115]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_4/x6_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W - W7 - W6 - W5, out_height + H_padding],  # Bottom right
    [W - W7 - W6 - W5, 0 + H_padding],           # Top right
    [W1 + W2 + W3 , 0 + H_padding],                       # Top left
    [W1 + W2 + W3 , out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_4 = cv2.warpPerspective(image, Homo, (int(W), int(H + H_padding*2)))

# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_4)



width:  2172  height:  300
denormalized_points:  [[1914.24  699.84]
 [1806.72  540.  ]
 [ 506.88  530.28]
 [ 422.4   718.2 ]]
Homography Matrix:  [[ 8.43360597e-01  1.16982651e+01  6.41381646e+03]
 [-7.60958762e-02  3.36220961e+00 -1.26304773e+03]
 [-1.79652157e-04  1.31133246e-03  1.00000000e+00]]


True

- Stream 5 ( Từ phải sang)

In [116]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_5/x7_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W - W7 - W6 - W5 - W4, out_height + H_padding],  # Bottom right
    [W - W7 - W6 - W5 - W4, 0 + H_padding],           # Top right
    [W1 + W2  , 0 + H_padding],                       # Top left
    [W1 + W2  , out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_5 = cv2.warpPerspective(image, Homo, (int(W), int(H + H_padding *2)))

# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_5)


width:  2722  height:  300
denormalized_points:  [[1795.2   694.44]
 [1708.8   525.96]
 [ 172.8   532.44]
 [  76.8   687.96]]
Homography Matrix:  [[ 3.80109711e+00  9.17206111e+00  3.75694332e+03]
 [ 4.37261560e-02  4.09985417e+00 -1.67482250e+03]
 [ 9.36674476e-05  1.31973569e-03  1.00000000e+00]]


True

- Stream 6 ( Từ phải sang)

In [117]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_6/x8_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W - W7 - W6 - W5 - W4 - W3, out_height + H_padding],  # Bottom right
    [W - W7 - W6 - W5 - W4 - W3, 0 + H_padding],           # Top right
    [W1  , 0 + H_padding],                       # Top left
    [W1   , out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_6 = cv2.warpPerspective(image, Homo, (int(W), int(H + H_padding*2)))

# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_6)



width:  2727  height:  300
denormalized_points:  [[1831.68  723.6 ]
 [1760.64  560.52]
 [ 245.76  509.76]
 [ 138.24  662.04]]
Homography Matrix:  [[ 2.90898365e+00  5.39186652e+00  9.00922578e+02]
 [-1.20695791e-01  3.86737246e+00 -1.45358257e+03]
 [-1.17881523e-05  1.23624302e-03  1.00000000e+00]]


True

- Stream 7 ( Từ phải sang)

In [118]:
img_path = "/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/stream_7/x9_frame_0.jpg"
folder_frame = img_path[:-14]
name = img_path.split('/')[-1].split('.')[0]

point_path = os.path.join(folder_frame, f"{name}.txt")

# Read the image
image = cv2.imread(img_path)
height, width = image.shape[:2]

# Read and denormalize the points
denormalized_points = read_points(point_path, width, height)

# Calculate the output size
out_width, out_height = calculate_output_size(denormalized_points)

print("width: ", out_width, " height: ", out_height)

# Define the destination points for the warp
# dst_points = np.array([
#     [out_width, out_height],  # Bottom right
#     [out_width, 0],           # Top right
#     [0, 0],                       # Top left
#     [0, out_height]               # Bottom left
# ], dtype=np.float32)

dst_points = np.array([
    [W - W7 - W6 - W5 - W4 - W3 - W2, out_height + H_padding],  # Bottom right
    [W - W7 - W6 - W5 - W4 - W3 - W2, 0 + H_padding],           # Top right
    [0  , 0 + H_padding],                       # Top left
    [0   , out_height + H_padding]               # Bottom left
], dtype=np.float32)

print("denormalized_points: ", denormalized_points)

# Calculate the homography matrix
Homo, _ = cv2.findHomography(denormalized_points, dst_points)
print("Homography Matrix: ", Homo)

# Save the homography matrix
matrix_array = np.array(Homo)
np.savetxt(os.path.join(folder_frame, 'matrix_homo.txt'), matrix_array)

# Warp the image
# warped_image = cv2.warpPerspective(image, Homo, (int(out_width), int(out_height)))
warped_image_7 = cv2.warpPerspective(image, Homo, (int(W), int(H + H_padding*2)))

# Save and show the warped image
output_path = os.path.join(folder_frame, "warped_frame.jpg")
cv2.imwrite(output_path, warped_image_7)


width:  2682  height:  300
denormalized_points:  [[1762.56  732.24]
 [1676.16  561.6 ]
 [ 155.52  507.6 ]
 [  53.76  657.72]]
Homography Matrix:  [[ 3.17510265e+00  2.15226846e+00 -1.58628346e+03]
 [-1.20289820e-01  4.14317832e+00 -1.57211779e+03]
 [ 4.04096525e-05  1.38144683e-03  1.00000000e+00]]


True

In [119]:
overview_image = np.zeros((H + H_padding *2, W, 3), dtype=np.uint8)

overview_image[:, :W1] = warped_image_7[:, :W1]
overview_image[:, W1:W1 + W2] = warped_image_6[:, W1:W1 + W2]
overview_image[:, W1 + W2:W1 + W2 + W3] = warped_image_5[:, W1 + W2:W1 + W2 + W3]
overview_image[:, W1 + W2 + W3 : W1 + W2 + W3 + W4] = warped_image_4[:,  W1 + W2 + W3 : W1 + W2 + W3 + W4]
overview_image[:, W1 + W2 + W3 + W4 : W1 + W2 + W3 + W4 + W5] = warped_image_3[:, W1 + W2 + W3 + W4 : W1 + W2 + W3 + W4 + W5]
overview_image[:,  W1 + W2 + W3 + W4 + W5 : W1 + W2 + W3 + W4 + W5 + W6] = warped_image_2[:,  W1 + W2 + W3 + W4 + W5 : W1 + W2 + W3 + W4 + W5 + W6]
overview_image[:, W1 + W2 + W3 + W4 + W5 + W6 : ] = warped_image_1[:, W1 + W2 + W3 + W4 + W5 + W6 : ]

cv2.imwrite("/home/sonlt373/Desktop/SoNg/Multi_cam_tracking/dev/multicam_tracking/data_wheel_daitu/overview_image.jpg", overview_image)






True